# Apa Itu Langchain ?

In [109]:
!pip install tavily-python langchain openai chromadb langchain-experimental pypdf langchain-openai langchainhub langchain-chroma

In [61]:
!export OPENAI_API_KEY = ''
!export TAVILY_API_KEY = ""

zsh:1: bad assignment
zsh:1: bad assignment


# Basic Usage

In [5]:
from langchain_openai import ChatOpenAI

In [6]:
llm = ChatOpenAI(model_name="gpt-4o-mini-2024-07-18", temperature=0)

In [8]:
resp = llm.invoke("What is the meaning of life?")

# LCEL

## Single Chain

In [28]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [29]:
prompt = ChatPromptTemplate.from_template(""" Anda adalah seorang guru matematika yang dapat menjelaskan suatu jawaban dari pertanyaan dengan
                                          baik kepada anak sekolah dasar. Jawablah dengan tepat dan penyampaian yang sesuai untuk
                                          anak sekolah dasar {pertanyaan}""")

chain = prompt | llm | StrOutputParser()

In [27]:
resp = chain.invoke({"pertanyaan": "Berapa jawaban dari 5*10/2 + 7"})
print(resp)

Baiklah, mari kita hitung bersama-sama!

Kita punya soal: \(5 \times 10 \div 2 + 7\).

Langkah pertama adalah melakukan perkalian dan pembagian dari kiri ke kanan. Jadi, kita mulai dengan \(5 \times 10\):

\[
5 \times 10 = 50
\]

Sekarang, kita ambil hasilnya dan bagi dengan 2:

\[
50 \div 2 = 25
\]

Sekarang kita sudah mendapatkan hasil sementara, yaitu 25. Selanjutnya, kita tambahkan 7:

\[
25 + 7 = 32
\]

Jadi, jawaban dari \(5 \times 10 \div 2 + 7\) adalah **32**. 

Bagaimana, mudah kan? Jika ada yang ingin ditanyakan lagi, silakan!


## Composed Chain

In [30]:
# Digunakan apabila ingin menggabungkan beberapa chain menjadi satu
# misal di sini saya ingin membuat sebuah flow di mana
# chain 1 : akan menggenerate list nama orang dan list aktivitas
# chain 2 : akan mengatur orang mana yang akan mengerjakan aktivitas yang mana

In [32]:
prompt_1 = ChatPromptTemplate.from_template(""" buatkan 10 nama orang dengan nama yang khas indonesia, kemudian berikan 10 kegiatan yang dapat dilakukan 
                                          pada topik berikut {topik}""")

chain_1 = prompt_1 | llm | StrOutputParser()

In [33]:
prompt_2 = ChatPromptTemplate.from_template(""" Berdasarkan nama-nama orang tersebut aturlah siapa yang akan melakukan aktivitas apa pada topik berikut {list}""")

chain_2 = {"list":chain_1} | prompt_2 | llm | StrOutputParser()

In [34]:
chain_2.invoke({"topik": "olahraga"})

'Berikut adalah pembagian aktivitas olahraga berdasarkan nama-nama orang yang telah disebutkan:\n\n1. **Senam Pagi Bersama**: **Dewi Anggraini** - Sebagai sosok yang energik dan peduli kesehatan, Dewi akan memimpin sesi senam pagi untuk meningkatkan kebugaran dan kebersamaan.\n\n2. **Turnamen Sepak Bola**: **Budi Santoso** - Dengan semangat tim yang tinggi, Budi akan mengorganisir turnamen sepak bola antar RT atau antar sekolah.\n\n3. **Lari Santai**: **Rizky Ramadhan** - Rizky yang aktif dan suka berolahraga akan memimpin acara lari santai di sekitar lingkungan.\n\n4. **Yoga di Alam Terbuka**: **Ayu Lestari** - Ayu yang memiliki ketertarikan pada kesehatan mental dan relaksasi akan mengadakan kelas yoga di taman atau pantai.\n\n5. **Bersepeda Bersama**: **Eko Susanto** - Eko yang suka berpetualang akan mengorganisir kegiatan bersepeda keliling kota atau desa.\n\n6. **Pelatihan Atletik**: **Andi Wijaya** - Andi yang memiliki pengalaman dalam olahraga akan mengadakan pelatihan atletik u

# Output Parser

## JsonOutputParser

In [35]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

In [39]:
# Misalkan kita memiliki biografi seorang tokoh yang terkenal, kemudian dari narasi biografi tersebut kita ingin mendapatkan informasi mengenai Nama,
# Tanggal Lahir, Tempat Lahir, dan Pekerjaan dari tokoh tersebut serta pengalamanya. maka kita dapat melakukannya seperti berikut

biography = """
LeBron James adalah seorang pemain bola basket profesional asal Amerika Serikat yang dianggap sebagai salah satu pemain terbaik dalam sejarah NBA. Ia lahir pada 30 Desember 1984, di Akron, Ohio. Sejak kecil, LeBron sudah menunjukkan bakat luar biasa dalam olahraga basket, yang membuatnya dikenal sebagai "anak ajaib" di dunia olahraga. Ia bersekolah di St. Vincent-St. Mary High School, di mana ia memimpin tim basket sekolah tersebut meraih kejuaraan negara bagian, sekaligus mendapatkan perhatian nasional sebagai pemain muda yang menjanjikan.

Pada tahun 2003, LeBron James memasuki NBA sebagai pilihan pertama dalam draft, direkrut oleh Cleveland Cavaliers. Kariernya langsung bersinar, dan ia dinobatkan sebagai NBA Rookie of the Year pada musim pertamanya. Ia membawa Cavaliers ke beberapa penampilan playoff dan mencapai Final NBA pada tahun 2007, meskipun mereka kalah dari San Antonio Spurs.

Pada 2010, LeBron membuat keputusan kontroversial dengan bergabung ke Miami Heat, di mana ia memenangkan dua gelar juara NBA pada tahun 2012 dan 2013. Setelah empat musim bersama Heat, ia kembali ke Cleveland Cavaliers pada 2014, dan pada 2016, LeBron berhasil membawa Cavaliers meraih gelar juara NBA pertama dalam sejarah klub, sekaligus memimpin comeback epik dari ketinggalan 1-3 melawan Golden State Warriors di Final NBA."""


# Define your desired data structure.
class Person(BaseModel):
    name: str = Field(description="Nama orang yang dimaksud")
    tempat_tanggal_lahir: str = Field(description="Tempat dan tanggal lahir orang tersebut")
    pekerjaan: str = Field(description="Pekerjaan dari orang tersebut")
    pengalaman: str = Field(description="Pengalaman dari orang tersebut")

In [41]:

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Person)

prompt = PromptTemplate(
    template="Buatkan berdasarkan biografi berikut.\n{biography}\n{format_instructions}\n",
    input_variables=["biography"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"biography": biography})

{'name': 'LeBron James',
 'tempat_tanggal_lahir': 'Akron, Ohio, 30 Desember 1984',
 'pekerjaan': 'Pemain Bola Basket Profesional',
 'pengalaman': 'Memasuki NBA pada tahun 2003 sebagai pilihan pertama dalam draft, dinobatkan sebagai NBA Rookie of the Year, memenangkan dua gelar juara NBA bersama Miami Heat pada tahun 2012 dan 2013, serta membawa Cleveland Cavaliers meraih gelar juara NBA pertama dalam sejarah klub pada tahun 2016.'}

# Memory

In [46]:
from operator import itemgetter
from typing import List

from langchain_openai.chat_models import ChatOpenAI

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.documents import Document
from langchain_core.messages import BaseMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field
from langchain_core.runnables import (
    RunnableLambda,
    ConfigurableFieldSpec,
    RunnablePassthrough,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

from typing import Optional

from langchain_community.chat_models import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

In [54]:
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

# Here we use a global variable to store the chat message history.
# This will make it easier to inspect it to see the underlying results.
store = {}

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [57]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "you are asistant for a physics teacher"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])
chain = prompt | llm | StrOutputParser()

In [58]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_by_session_id,
    input_messages_key="question",
    history_messages_key="history",
)

chain_with_history.invoke(
    {"question": "What is gravity?"},
    config={"configurable": {"session_id": "user_abdul"}}
)

"Gravity is a fundamental force of nature that attracts two bodies with mass toward each other. It is responsible for a variety of phenomena, including the falling of objects to the ground, the orbits of planets around stars, and the formation of galaxies.\n\nIn classical physics, gravity is described by Isaac Newton's law of universal gravitation, which states that every point mass attracts every other point mass with a force that is directly proportional to the product of their masses and inversely proportional to the square of the distance between their centers. The formula for this force \\( F \\) is given by:\n\n\\[\nF = G \\frac{m_1 m_2}{r^2}\n\\]\n\nwhere:\n- \\( F \\) is the gravitational force,\n- \\( G \\) is the gravitational constant (\\(6.674 \\times 10^{-11} \\, \\text{N(m/kg)}^2\\)),\n- \\( m_1 \\) and \\( m_2 \\) are the masses of the two objects,\n- \\( r \\) is the distance between the centers of the two masses.\n\nIn modern physics, gravity is described by Albert Ein

# Use Google Search Retriever

In [110]:
from langchain_community.tools import TavilySearchResults

retriever = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    # include_images=True,
)
# tool_resp = tool.invoke({"query": "Apa yang terjadi pada wasit bahrain pada pertandingan AFC indonesia vs bahrain"})
# tool_resp[0]["content"]

In [96]:
# alternatif untuk tavily search
# from langchain_community.retrievers import TavilySearchAPIRetriever
# retriever = TavilySearchAPIRetriever(
#     k=3,
#     search_depth="advanced")

In [111]:
prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

Context: {context}

Question: {question}"""
)

def format_docs(docs):
    return "\n\n".join(doc["content"] for doc in docs)


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = "Apa hal kontroversial yang dilakukan wasit pada pertandingan bahrain vs indonesia?"
chain.invoke(query)

'Hal kontroversial yang dilakukan wasit Ahmed Al Kaf pada pertandingan Bahrain vs Indonesia adalah membiarkan tambahan waktu berlangsung melebihi dari yang diumumkan, yaitu dari 6 menit menjadi 9 menit. Keputusan ini memungkinkan Tim Bahrain mencetak gol penyama kedudukan pada menit 90+9, yang berkontribusi pada hasil imbang 2-2.'